In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# pre-trained model from drive
!mkdir models
!cp gdrive/My\ Drive/models/bestmodel_tune_itos.pkl models
!cp gdrive/My\ Drive/models/bestmodel_tune.pth models

In [0]:
from fastai.text import *

from tqdm import tqdm_notebook

import torch
torch.cuda.set_device(0)

In [0]:
path = Path('.')
bs = 64

In [0]:
texts = pd.read_csv('jokes_extended_vk_anekdot_preproc.csv', index_col=0)
texts.dropna(inplace=True)

In [0]:
data = TextList.from_df(texts, 
                        processor=[TokenizeProcessor(tokenizer=Tokenizer(lang="xx")), 
                                                     NumericalizeProcessor(min_freq=2, max_vocab=60000)])\
                                                    .split_by_rand_pct(.1)\
                                                    .label_for_lm()\
                                                    .databunch(bs=bs)

In [0]:
learn = language_model_learner(data=data, arch=AWD_LSTM, pretrained=None)
learn.load_pretrained('models/bestmodel_tune.pth', 'models/bestmodel_tune_itos.pkl')

55329

In [0]:
def generate(learn, n_words=200, temperature=1.0):
  raw_data = learn.predict(text='', n_words=n_words, temperature=temperature)

  jokes, joke = [], ''
  for word in raw_data.split(' '):
    if word == 'xxbos':
      jokes.append(joke)
      joke = ''
    else:
      joke += word + ' '


  for i, joke in enumerate(jokes):
    print(f'Шутка {i+1}\n{joke}\n')

In [0]:
generate(learn, n_words=400)

Шутка 1
 идеальный порядок - это российский мир , у которого британия богатая . если вы хотите , чтобы в мире был главный донором коррупции , то получится умер ! 

Шутка 2
анекдоты про папу 
 
  - вовочка , а ты точно дебил 
  - нет , сережа , девочку в школе примут . 
  - ну , и пошла на хй ! 
  - помнится ребенок , наверное , дядя у отца сам был . 
  : .- 

Шутка 3
школьник школы , который , обнял николая валуева , внимательно смотрел на итальянский флаг . увидев крики ветеранов человечества , подумал : чувак : " сядь - ка ночи володя : заметив что это такое : взрослые стали пропадать к нам овны , тельцы , козероги . но придумали такое же самое : мишки доктором в разрешили эль среднюю грудь , а в бассейн постель кладут ! 

Шутка 4
а вы ему забыли жениться на еврейке купил у него крестным кредиторам . 

Шутка 5
похоже женщина люся , смотри , козлы . мы не перебью её так , как будто вы туда попали . 

Шутка 6
после провала " евровидения " на евровидении судьи назовите все места для отб